In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_id = "watt-ai/watt-tool-8B"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype='auto', device_map="auto")


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [2]:

# Example usage (adapt as needed for your specific tool usage scenario)

# # You should only return the function call in tools call sections.

# # If you decide to invoke any of the function(s), you MUST put it in the format of [func_name1(params_name1=params_value1, params_name2=params_value2...), func_name2(params)]
# # You SHOULD NOT include any other text in the response.
# # Here is a list of functions in JSON format that you can invoke.\n{functions}\n
# # """
# User query


tools = [
    {
        "name": "finding_summarizer",
        "description": "Findings Summarizer skill is built for summarizing or in other words explaining the meaning of findings found in an investigation. User query may have the words 'finding' or 'summerize' or 'summary' in it.",
        "examples": ["summerize this incident", "write a summary of this incident", "summerize findings", "explain the findings", "list findings", "explain the detections that triggered this finding", "what is the meaning of this incident", "Can you simplify this (json)"],
        "parameters": {
            "type": "object",
            "properties": {
                "user_input": {
                    "type": "string",
                    "description": "User query asking a summary of the incident. User query has the word summary or summarize or findings in it."
                },
                "output": {
                    "type": "string",
                    "description": "A natural language summary of the incident."
                }
            },
            "required": []
        }
    },
    {
        "name": "investigation_report",
        "description": "Investigation report skill is built to generate a report for the investigation including key information that gives a mid to high level overview of the incident. It gives a chronological timeline of events associated with the incident including the dates, times and descriptions of activity for investigation creation, finding discovery and notes being added by the analyst. User query will have the word 'report' in it.",
        "examples": ["write a report", "Generate a report", "Give me timeline of events","Provide a timeline of key events from analyst notes", "report", "make an investigation report", "generate final investigation report for incident"],
        "parameters": {
            "type": "object",
            "properties": {
                "user_input": {
                    "type": "string",
                    "description": "User query asking to generate a report.  User query has the word 'report' in it."
                },
                "output": {
                    "type": "string",
                    "description": "A splunk incident report."
                }
            },
            "required": []
        }
    },
    {
        "name": "conversation_response",
        "description": "Takes a user's request which is not handled by ['finding_summarizer', 'investigation_report', 'spl_writer'] and responds with security relevant information and pertinent information already in the chat history. User may ask questions about the investigation data.",
        "examples": ["as9kx", "why do we exist?", "Hi how are you?", "Are you an AI?", "I see you are a bot", "explain splunk", "what can you do with spl", "3 + 3", "what are your skills?", "help", "Are there any MITRE in this investigation", "explain the detections that triggered this finding", "give the number of findings in last investigation"],
        "parameters": {
            "type": "object",
            "properties": {
                "user_input": {
                    "type": "string",
                    "description": "User query with general question or question about value of specific data in the investigation. If user's query doesn't match any other function, pick conversation_response."
                },
                "output": {
                    "type": "string",
                    "description": "Response answering the user question."
                }
            },
            "required": []
        }
    },
    {
        "name": "spl_writer",
        "description": "User input is either natural language or SPL query. Converts a natural language request into an SPL query that can be executed to fulfill the user's intent. User input can be an ask for data analysis, show data insights, creation of data entries, find information from data, data manipulation action, monitor activity, replace data, extract or give entities, show or display or get events/entities, extracting event data from last 24 hours or a time window, or an ask to write code. User input may also ask to modify the previous query or add something to previous query.",
        "examples": ["List available indices", "your previous SPL did not work","Create a record with a value of 'abcd123'","get me all risk entities related to this incident", "give me another query for this task", "top 10 IP addresses by count"," all error messages in the logs from yesterday","Show me all the notables for this host","average daily ingestion per month", "previous SPL is wrong","show me all notables for this host in the last 60 days","Find accounts associated with failed login attempts.","check _key field in lookup","INFO  CMBucket  Freezing bid","list all the hostnames in my deployment","who are the users associated with login attempts for linux in past 24 hours","list sources that are consuming a lot of GPU RAM","search aws access logs last 3 errors","show me all the indexes that do not have any logs", "get risk events in last 24 hours", "give me users associated with risk event in past 24 hours", "replace the risk object", "add user=ax to the last spl query", "update the search to look for id=123"],
        "parameters": {
            "type": "object",
            "properties": {
                "intent": {
                    "type": "string",
                    "description": "A natural language description of the user's intent or request. User may ask to modify or add to past SPL query."
                },
                "output": {
                    "type": "string",
                    "description": "The SPL query that corresponds to the user's request."
                }
            },
            "required": ["intent"]
        }
    }
]
def out_sys_prompt(tools):
    return f"""
    You are an expert in picking functions. You are given a question/statement and a set of possible functions.
    Based on the question/statement, you will need to make one function to achieve the purpose in question/statement.
    You do not provide any explanation, additional text, or answers to user's input as your output. 
    If the input is correct or incorrect SPL query, pick the function spl_writer. 
    If the input asks you to create data entries, find from data, do data manipulation or data parsing, show or display data insights, write code, update or add to previous SPL query or asks if the past query was incorrect, pick the function spl_writer. 
    If the input is gibbrish, pick the function conversation_response. 
    AS A STRICT RULE, do not answer any of the user questions and STRICTLY only output the function for the user's query from this list of functions:investigation_report, conversation_response, spl_writer, finding_summarizer"
    STRICTLY follow this output format for your response:
    Example Query: generate report
    Your output: investigation_report
    Strictly only output the picked function name without any additional text.
    Here is a list of functions in JSON format that you can invoke. {tools}\n
    """
prompt=out_sys_prompt(tools)
#system_prompt.format(functions=tools)

In [13]:

# Example usage (adapt as needed for your specific tool usage scenario)

# # You should only return the function call in tools call sections.

# # If you decide to invoke any of the function(s), you MUST put it in the format of [func_name1(params_name1=params_value1, params_name2=params_value2...), func_name2(params)]
# # You SHOULD NOT include any other text in the response.
# # Here is a list of functions in JSON format that you can invoke.\n{functions}\n
# # """
# User query


tools = [
    {
        "name": "finding_summarizer",
        "description": "Findings Summarizer skill is built for summarizing or in other words explaining the meaning of findings found in an investigation. User query may have the words 'finding' or 'summarize' or 'summary' in it.",
        "examples": ["summarize this incident", "write a summary of this incident", "summarize findings", "explain the findings", "list findings", "explain the detections that triggered this finding", "what is the meaning of this incident", "Can you simplify this (json)"],
        "parameters": {
            "type": "object",
            "properties": {
                "user_input": {
                    "type": "string",
                    "description": "User query asking a summary of the incident. User query may have the word summary or summarize or findings in it."
                },
                "output": {
                    "type": "string",
                    "description": "A natural language summary of the incident."
                }
            },
            "required": []
        }
    },
    {
        "name": "investigation_report",
        "description": "Investigation report skill is built to generate a report for the investigation including key information that gives a mid to high level overview of the incident. It gives a chronological timeline of events associated with the incident including the dates, times and descriptions of activity for investigation creation, finding discovery and notes being added by the analyst. User query may have the word 'report' in it.",
        "examples": ["write a report", "Generate a report", "Give me timeline of events","Provide a timeline of key events from analyst notes", "report", "make an investigation report", "generate final investigation report for incident"],
        "parameters": {
            "type": "object",
            "properties": {
                "user_input": {
                    "type": "string",
                    "description": "User query asking to generate a report.  User query may have the word 'report' in it."
                },
                "output": {
                    "type": "string",
                    "description": "A splunk incident report."
                }
            },
            "required": []
        }
    },
    {
        "name": "conversation_response",
        "description": "Takes a user's request which is not handled by ['finding_summarizer', 'investigation_report', 'spl_writer'] and responds with security relevant information and pertinent information already in the chat history. User may ask questions about the investigation data.",
        "examples": ["as9kx", "why do we exist?", "Hi how are you?", "Are you an AI?", "I see you are a bot", "explain splunk", "what can you do with spl", "3 + 3", "what are your skills?", "help", "what is the time of first finding?", "explain the detections that triggered this finding", "what is the status of this investigation?", "give the number of findings in last investigation"],
        "parameters": {
            "type": "object",
            "properties": {
                "user_input": {
                    "type": "string",
                    "description": "User query with general question or question about value of specific data in the investigation. If user's query doesn't match any other function, pick conversation_response."
                },
                "output": {
                    "type": "string",
                    "description": "Response answering the user question."
                }
            },
            "required": []
        }
    },
    {
        "name": "spl_writer",
        "description": "User input is either natural language or SPL query. Converts a natural language request into an SPL query that can be executed to fulfill the user's intent. User input can be an ask for data analysis, show data insights, creation of data entries, find information from data, data manipulation action, monitor activity, replace data, extract or give entities, show or display or get events/entities, extracting event data from last 24 hours or a time window, or an ask to write code. User input may also ask to modify the previous query or add something to previous query.",
        "examples": ["List available indices", "your previous SPL did not work","Create a record with a value of 'abcd123'","get me all risk entities related to this incident", "give me another query for this task", "top 10 IP addresses by count"," all error messages in the logs from yesterday","Show me all the notables for this host","average daily ingestion per month", "previous SPL is wrong","show me all notables for this host in the last 60 days","Find accounts associated with failed login attempts.","check _key field in lookup","INFO  CMBucket  Freezing bid","list all the hostnames in my deployment","who are the users associated with login attempts for linux in past 24 hours","list sources that are consuming a lot of GPU RAM","search aws access logs last 3 errors","show me all the indexes that do not have any logs", "get risk events in last 24 hours", "give me users associated with risk event in past 24 hours", "replace the risk object", "add user=ax to the last spl query", "update the search to look for id=123"],
        "parameters": {
            "type": "object",
            "properties": {
                "intent": {
                    "type": "string",
                    "description": "A natural language description of the user's intent or request. User may ask to modify or add to past SPL query."
                },
                "output": {
                    "type": "string",
                    "description": "The SPL query that corresponds to the user's request."
                }
            },
            "required": ["intent"]
        }
    }
]
def out_sys_prompt(tools):
    return f"""
    You are an expert in picking functions. You are given a question/statement and a set of possible functions.
    Based on the question/statement, you will need to make one function to achieve the purpose in question/statement.
    You do not provide any explanation, additional text, or answers to user's input as your output. 
    If the input is correct or incorrect SPL query, pick the function spl_writer. 
    If the input asks you to create data entries, find from data, do data manipulation or data parsing, show or display data insights, write code, update or add to previous SPL query or asks if the past query was incorrect, pick the function spl_writer. 
    If the input is gibbrish, pick the function conversation_response. 
    AS A STRICT RULE, do not answer any of the user questions and STRICTLY only output the function for the user's query from this list of functions:investigation_report, conversation_response, spl_writer, finding_summarizer"
    STRICTLY follow this output format for your response:
    Example query: generate report
    Your output: investigation_report
    Strictly only output the picked function name without any additional text.
    Here is a list of functions in JSON format that you can invoke. {tools}\n
    """
prompt=out_sys_prompt(tools)
#system_prompt.format(functions=tools)

In [3]:
query = "how many findings are in this investigation"

messages = [
    {'role': 'system', 'content': prompt},
    {'role': 'user', 'content': query}
]
inputs = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt").to(model.device)

outputs = model.generate(inputs, max_new_tokens=512, do_sample=False, num_return_sequences=1, eos_token_id=tokenizer.eos_token_id)
print(tokenizer.decode(outputs[0][len(inputs[0]):], skip_special_tokens=True))


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


conversation_response


In [4]:
# load orchestrator data
import pandas as pd
data=pd.read_csv("../../grading/data/orchestrator_final_main.csv")
print(data.head())
labels = data['Human_label']
input_data = data['latest_msg']

for i in range(len(labels)):
    if labels[i]=="summerize_findings":
        labels[i]="finding_summarizer"
    if labels[i]=="findings_summerizer":
        labels[i]="finding_summarizer"  
    if labels[i]=="finding_summary":
        labels[i]="finding_summarizer"
    # if labels[i]=="spl or conversation?":
    #     labels[i]="spl_writer"
    # if labels[i]=="spl writer or conversation?":
    #     labels[i]="spl_writer"
responses = []
graph_latency = []
response_latency = []
unique_lab=sorted(set(labels))
print(unique_lab)

label_mapping = {value: idx for idx, value in enumerate(unique_lab)}
print(label_mapping)
attention_graphs = []

# model = llm()
limit = len(data)
del data

   Unnamed: 0.1  Unnamed: 0  \
0             0           0   
1             1           1   
2             2           2   
3             3           3   
4             4           4   

                                           messsages  \
0  [{'role': 'system', 'content': "You are a tech...   
1  [{'role': 'system', 'content': "You are a tech...   
2  [{'role': 'system', 'content': "You are a tech...   
3  [{'role': 'system', 'content': "You are a tech...   
4  [{'role': 'system', 'content': "You are a tech...   

                        latest_msg            Human_label  \
0                              Hi!  conversation_response   
1            What are your skills?  conversation_response   
2                  What is splunk?  conversation_response   
3              What does spl mean?  conversation_response   
4  What is the weather like today?  conversation_response   

     reference_skill_gpt data_source                              llama31_70b  \
0  conversation_response    

/tmp/ipykernel_28116/3905135648.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  labels[i]="finding_summarizer"
/tmp/ipykernel_28116/3905135648.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  labels[i]="finding_summarizer"
/tmp/ipykernel_28116/3905135648.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  labels[i]="finding_summarizer"
/tmp/ipykernel_28116/3905135648.py:12: SettingWithCopyWarning: 
A value is trying to be set

In [5]:
# inference
from time import time
responses=[]
latency=[]
for i in range(len(input_data)):
    print(i, input_data[i])
    query = input_data[i]
    
    messages = [
        {'role': 'system', 'content': prompt},
        {'role': 'user', 'content': query}
    ]
    start = time()
    inputs = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt").to(model.device)

    outputs = model.generate(inputs, max_new_tokens=512, do_sample=False, num_return_sequences=1, eos_token_id=tokenizer.eos_token_id)
    tmp=tokenizer.decode(outputs[0][len(inputs[0]):], skip_special_tokens=True)
    end= time()
    responses.append([tmp])
    latency.append(end-start)
    print("resp",tmp)
    print("orig",labels[i])
    
    

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


0 Hi!


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


resp conversation_response
orig conversation_response
1 What are your skills?


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


...

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


resp conversation_response
orig conversation_response
53 can you recommend next steps


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


resp conversation_response
orig conversation_response
54 lookup all activity of this affected asset


KeyboardInterrupt: 

In [7]:
print(len(latency))
import numpy as np
np.mean(np.array(latency))

54


np.float64(2.5881452427970038)

In [17]:
def check_strings(response):
    keywords=["investigation_report", "conversation_response", "spl_writer", "finding_summarizer"]
    result=[]
    for item in response:
        found=[kw for kw in keywords if kw in str(item)]
        if len(found)>1:
            result.append(["conversation_response"][0])
        elif found:
            result.append(found[0])
        else:
            result.append(["conversation_response"][0])
    return result
            
        
    
    

In [18]:
responses_parsed=check_strings(responses)
print(responses_parsed)


['conversation_response', 'conversation_response', 'conversation_response', 'conversation_response', 'conversation_response', 'conversation_response', 'conversation_response', 'conversation_response', 'conversation_response', 'finding_summarizer', 'conversation_response', 'investigation_report', 'conversation_response', 'conversation_response', 'conversation_response', 'conversation_response', 'spl_writer', 'spl_writer', 'spl_writer', 'spl_writer', 'conversation_response', 'spl_writer', 'spl_writer', 'spl_writer', 'spl_writer', 'spl_writer', 'finding_summarizer', 'finding_summarizer', 'finding_summarizer', 'finding_summarizer', 'finding_summarizer', 'finding_summarizer', 'finding_summarizer', 'finding_summarizer', 'finding_summarizer', 'finding_summarizer', 'investigation_report', 'investigation_report', 'investigation_report', 'investigation_report', 'investigation_report', 'spl_writer', 'spl_writer', 'finding_summarizer', 'conversation_response', 'spl_writer', 'spl_writer', 'spl_writ

In [19]:
true_labels=[]
for i in range(len(labels)):
    true_labels.append(labels[i])
               

In [20]:

def get_acc(extracted_labels,true_labels):
    correct=sum(set(pred)==set(actual) for pred,actual in zip(extracted_labels,true_labels))
    return correct/len(true_labels) 
print(get_acc(responses_parsed,true_labels))

0.9482758620689655


In [21]:
for i in range(len(true_labels)):
    if true_labels[i] != responses_parsed[i]:
        print(i)
        print(responses_parsed[i],input_data[i])
        print(true_labels[i])

20
conversation_response Search for security investigations like multiple login failures from a single IP address.
spl_writer
60
conversation_response show me all the risk events for this investigation
spl_writer
74
conversation_response what spl could i use to filter these results for commdline that used the curl utility
spl_writer
88
conversation_response Why didnt you generate an SPL
spl_writer
117
conversation_response Has this entity been part of other investigations recently ?
spl_writer
119
conversation_response what findings are linked to this investigation
finding_summarizer
134
finding_summarizer how many findings are in this incident
conversation_response
138
conversation_response get all risk objects listed in this investigation
conversation_response or spl_writer
151
conversation_response What are the MITRE ATT&CK techniques associated with this finding?
conversation_response or findings_summarizer


In [12]:
# save
from pathlib import Path
output_directory="../../grading/data/v2"
pd.DataFrame({
    "responses": responses,
    "parsed_responses":responses_parsed,
    "label":true_labels,
    "inputs":input_data,
    "latency":latency
    

     }).to_csv(Path(output_directory, "responses_new_watt.csv"), index=False, header=True)

In [68]:
len(latency)

174